In [1]:
from bs4 import BeautifulSoup, Comment
from urllib.request import urlopen
import pandas as pd
from selenium import webdriver
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from random import *
from datetime import timedelta, date

In [ ]:
raw_data = {'title':[],'date':[],'url':[],'script':[]}
rr = randrange(3)
    
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


start_date = date(2017, 1, 4)
end_date = date(2018, 1, 1)        
        
for day in daterange(start_date, end_date):
    try:
        
        df = pd.DataFrame(raw_data)

        driver = webdriver.Chrome("chromedriver.exe")
        driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B8%88%EB%A6%AC&nso=so%3Add%2Cp%3Afrom" + day.strftime("%Y%m%d") + "to" + day.strftime("%Y%m%d") + "%2Ca%3Aall0")
        driver.set_page_load_timeout(30)

        driver.find_element_by_xpath("""//*[@id="news_popup"]/a""").click()
        time.sleep(rr)

        driver.find_element_by_xpath("""//*[@id="ca_2227"]""").click()
        time.sleep(rr)

        driver.find_element_by_xpath("""//*[@id="_nx_option_media"]/div[2]/div[3]/button[1]/span""").click()
        time.sleep(rr)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        a_tags = soup.find_all('a', class_='_sp_each_title')

        tmp = []
        tmp.append(a_tags)


        page = driver.find_elements_by_xpath("""//div[@class="paging"]//a""")
        time.sleep(rr)

        for i in range(1, len(page)):

            driver.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B8%88%EB%A6%AC&nso=so%3Add%2Cp%3Afrom" + day.strftime("%Y%m%d") + "to" + day.strftime("%Y%m%d") + "a:all&mynews=1&cluster_rank=60&start=" + str(i*10+1) + "&refresh_start=0")
            driver.set_page_load_timeout(30)



            soup = BeautifulSoup(driver.page_source, 'html.parser')
            a_tags = soup.find_all('a', class_='_sp_each_title')
            tmp.append(a_tags)
            time.sleep(rr)


        date=''
        prepro=''
        cnt= 1

        for k in range(len(tmp)):
            for j in tmp[k]:
                #제목
                print("제목 : " + j.get_text())
                title = j.get_text()
                #URL
                url = str(j['href'])
                driver.get(url)
                time.sleep(rr)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
             #   print("URL : " + url)
               #날짜
                if soup.find('div', class_='info-text') is not None:
                    date = soup.find('div', class_='info-text').find_all('li')
                    date = str(date[1].text)
                    date = date[4:14]
             #       print("date : "+ date)
                #본문
                if soup.find_all('div', id='article-view-content-div') is not None:
                    div_tag = soup.find_all('div', id='article-view-content-div')
                    prepro = str(div_tag)
                    prepro = re.sub('\n','',prepro,0).strip()
                    prepro = re.sub('<!--.+?-->','',prepro,0).strip()
                    prepro = re.sub('<.+?>','',prepro,0).strip()
          #          print(prepro)
                new_df = pd.DataFrame({'title':title, 'date':date, 'url':url, 'script':prepro}, index=[0])
                df = pd.concat([df, new_df])
        #        print('geeeee')
                driver.close()
                driver = webdriver.Chrome("chromedriver.exe")

            df.to_csv("infomax" + day.strftime("%Y-%m-%d") + ".txt", header=True, index=False)

    #    driver.close()
    #    driver = webdriver.Chrome("chromedriver.exe")
    except:
        print('ERROR')
        new_df = pd.DataFrame({'title':title, 'date':date, 'url':url, 'script':prepro}, index=[0])
        df = pd.concat([df, new_df])
        df.to_csv("infomax" + day.strftime("%Y-%m-%d") + ".txt", header=True, index=False)
        alert = driver.switch_to_alert()
        alert.accept()
        continue